    Customer orders:

In [1]:
#Connection check: status - complete

import mysql.connector

mydb = mysql.connector.connect( user='root', password='abcd',
                                host='127.0.0.1', database='shopkeeps',
                               auth_plugin='mysql_native_password')

if (mydb):
    print('Connection successful')
else:
    print('Connections failed')

Connection successful


In [3]:
# Check for item : status - complete
def item_exists(item):
    cursor=mydb.cursor()
   
    cursor.execute('select iname from itemmaster')
    items=cursor.fetchall()
    
    if (item,) in items:
        return(1)
            
    else:
        return(0)
#item_exists('bread')

In [4]:
# Check for order : status - complete
def order_exists(ordercode):
    cursor=mydb.cursor()
   
    cursor.execute('select ordercode from orders')
    ordercodes=cursor.fetchall()
    
    if (ordercode,) in ordercodes:
        return(1)
            
    else:
        return(0)
#order_exists('')

In [5]:
# ordercode generator : status - completed
def ocode_gen():
    import random
    import string
 
    def random_string_generator(str_size):
        return ''.join(str(random.randint(0,9)) for x in range(str_size)) 
    size = 7
    ocode='OD'+random_string_generator(size)
    
    cursor=mydb.cursor()
    
    cursor.execute('select ordercode from orders')
    codes=cursor.fetchall()
    if (ocode,) in codes:
        ocode_gen()
    else: 
        return(ocode)
#print(ocode_gen())

In [6]:
# order search : status - complete
def order_search(code):
    cursor=mydb.cursor()
    exists=order_exists(code)
    if exists==1:
        cursor.execute('select * from orders where ordercode="'+code+'"')
        print(cursor.fetchone())
    else:
        print('Does not exist')
#order_search('')

In [7]:
#Order Removal : status - complete
def order_removal(ordercode):
    cursor=mydb.cursor()
   
    exists=order_exists(ordercode)
    
    if exists==1 :
        cursor.execute('select id from orders where ordercode="'+ordercode+'"')
        id=cursor.fetchone()[0]
        print(id)
        cursor.execute('delete from ordetails where oid='+str(id))
        cursor.execute('delete from orders where id='+str(id))
        print('Order cancelled')
       
    else:
        print('Order does not exist')
        
    mydb.commit()
#order_removal('RRlvPhfJ')

In [8]:
#Stock available : status - 
def stock_available(iname):
    if item_exists(iname)==1:
        cursor=mydb.cursor()
        cursor.execute('select stock from itemmaster where iname="'+iname+'"')
        x=cursor.fetchone()[0]
        
        if x>0:
            return(1)
        else:
            return(0)
    else:
        return(0)
#stock_available('pen')

In [9]:
#total price : status - complete
def tprice(ordercode):
    cursor=mydb.cursor()
    cursor.execute('select id from orders where ordercode="'+ordercode+'"')
    oid=cursor.fetchone()[0]
    cursor1=mydb.cursor()
    cursor1.execute('select sum(price) from ordetails where oid="'+str(oid)+'"')
    tprice=cursor1.fetchone()[0]
    return(tprice)
#print(tprice(''))

In [10]:
# New order : status -
def add_order(items,description=''):
    cursor=mydb.cursor()    
    ordercode=ocode_gen()
    cursor.execute('insert into orders(ordercode,description,tprice) values("'+ordercode+'","'+description+'",0)')
    cursor.execute('select id from orders where ordercode="'+ordercode+'"')
    oid=cursor.fetchone()[0]

    for item in items.keys():
        if item_exists(item)==1:
            cursor.execute('select stock from itemmaster where iname="'+item+'"')
            stock=cursor.fetchone()[0]
            i=0
            qty=items[item]
            
            if stock_available(item)==1 and stock-qty>=0:
                i=i+1
                cursor.execute('select id,price from itemmaster where iname="'+item+'"')
                fetch=cursor.fetchone()
                itno=fetch[0]
                price=fetch[1]
                print(fetch,itno,price,qty)
                cursor.execute('insert into ordetails(oid,itno,p_unit,qty,price) values('+str(oid)+','+str(itno)+','+str(price)+','+str(qty)+','+str(qty*price)+')')
                cursor.execute('update itemmaster set stock='+str(stock-qty)+' where iname="'+item+'"')
                cursor.execute('update itemmaster set sold='+str(qty)+' where iname="'+item+'"')

                
            else:
                print('Not enough stock. Stock left: ',stock)
        else: 
            print('item does not exist')
    if i>0:
        mydb.commit()
        totalprice=tprice(ordercode)
        cursor.execute('update orders set tprice='+str(totalprice)+' where ordercode="'+str(ordercode)+'"')
        mydb.commit()
        print('Order placed')
        print('Amount to be payed:',totalprice)
    
add_order(items={'dal':2,'rice':10},description='make it quick')
    

(3, 80.0) 3 80.0 2
(2, 100.0) 2 100.0 10
Order placed
Amount to be payed: 1160.0


In [11]:
# Order completion check : status - complete
def ord_complete(id):
    cursor=mydb.cursor()
    cursor.execute('update orders set status="complete" where id='+str(id))
    mydb.commit()
    print('Order complete')
#ord_complete(10)    

In [12]:
# Array of orders: status - complete
def pending_orders():
    cursor=mydb.cursor()
    cursor.execute('select id,ordercode from orders order by id')
    y=0
    for x in cursor.fetchall():
        if y!=5:
            print(x)
            y=y+1
        else: 
            break
#pending_orders()

In [13]:
# display order: status - complete
def order_display(id):
    cursor=mydb.cursor()
    cursor.execute('select s.iname , p.qty , p.price from itemmaster s, ordetails p where p.oid=10 and s.id=p.itno;')
    for item in cursor.fetchall():
        print(item)
#order_display(12)

In [14]:
#Connection check: status - complete

import mysql.connector

mydb = mysql.connector.connect( user='root', password='password',
                                host='127.0.0.1', database='shopkeeps',
                               auth_plugin='mysql_native_password')

if (mydb):
    print('Connection successful')
else:
    print('Connections failed')
    
    

#item_exists(item)
#order_exists(ordercode)
#ordercode_gen()
#order_search(ordercode)
#order_removal(ordercode)
#stock_available(item)
#tprice(ordercode)
#add_order(item_dict,description)
#ord_complete(id)
#pending_orders()
#order_display(id)


Connection successful
